In [1]:
from hashlib import sha256
from math import gcd
from random import randint
from Crypto.Util.number import getPrime, inverse
import numpy as np

# Import lll
#from lll import lll

def key_gen():
    phi = 2
    e = 2
    while gcd(phi, e) != 1 : 
        p = random_prime(2**1024, proof = False)
        q = random_prime(2**1024, proof = False)
        n = p*q
        phi = (p-1) * (q-1)
        e = 65537
    d = inverse_mod(e, phi)
    return (e, d, n, p, q)

def sign(m, d, p, q, n, bug = False):
    dp = d % (p-1)
    dq = d % (q-1)
    h = int.from_bytes(sha256(m).digest(), byteorder = "big")

    if bug:
        sp = randint(1, p - 1)
    else:
        sp = power_mod(h, dp, p)

    sq = power_mod(h, dq, q)
    signature = crt([sp, sq], [p, q])

    # PKCS#1 V1.5 signature
    prefix_sha256 = b'\x30\x31\x30\x0d\x06\x09\x60\x86\x48\x01\x65\x03\x04\x02\x01\x05\x00\x04\x20'
    pad_len = 256 - len(h.to_bytes(32, byteorder="big")) - len(prefix_sha256) - 3
    pad = b'\x00\x01' + b'\xff' * pad_len + b'\x00' + prefix_sha256 + h.to_bytes(32, byteorder="big")

    print("Signature (int):", signature)
#     print("Signature (hex):", signature.to_bytes(256, byteorder="big").hex())
#     print("Padding:", pad.hex())

    return signature, pad

def verify(m, s, e, n):
    decrypted = power_mod(s, e, n)
    decrypted_bytes = int(decrypted).to_bytes(256, byteorder="big")

    # Extraction du hash et comparaison
    h_calculated = sha256(m).digest()
    h_extracted = decrypted_bytes[-32:]

    print("Calculated hash from m:", h_calculated.hex())
    print("Extracted hash of pad :", h_extracted.hex())

    return h_calculated == h_extracted


def attack(s, e, n, pad):

    # Known part of the padding
    a = pad[:-32]
    a_int = int.from_bytes(a, byteorder='big')
    
    n1 = (pow(s, e) - a_int) % n

    # Construct the matrix
    log_r = 256
    B = Matrix(ZZ, 3, 3)  # Crée une matrice 3x3 avec des entiers de SageMath
    B[0, 0] = -2**(2 * log_r)
    B[0, 1] = 2**log_r * n1
    B[0, 2] = 0
    B[1, 0] = 0
    B[1, 1] = -2**log_r
    B[1, 2] = n1
    B[2, 0] = 0
    B[2, 1] = 0
    B[2, 2] = n
    
    # Réduction LLL sur la matrice
    reduced_matrix = B.LLL()

    # Affichage de la matrice réduite
    print("Matrice réduite :", reduced_matrix)
    
    print("-------------------")
    
    for rows in reduced_matrix:
        # v = reduced_matrix[i]  # Prendre la première ligne de la matrice réduite
        r1 = rows[2]  # On suppose que r_1 est dans la dernière colonne après réduction

        # Calcul de p à partir du gcd
        p = gcd(n, n1 - r1)

        if p != 1 :
            print("Attack success !!!")
            print("n1 : ", n1)
            print("r1 : ", r1)
            print("p : ", p)
            break


(e, d, n, p, q) = key_gen()
m = b"This message is signed with RSA-CRT!"
(s, pad) = sign(m, d, p, q, n, True)
# print("e = %s" % str(e))
# print("n = %s" %str(n))
# print("s = %s" %s)

print("Verify signature: ", verify(m, s, e, n))

print("-------------------")
print("Attack")
print("-------------------")
attack(s, e, n, pad)

Signature (int): 43418496953363085250058497905590098850291671619354810107380538936343339244666472141310889102884691729916601903920406453725658620929547802325747810287049109700187048871496775263277926147935939039464347145163903614304865007339741766839009307299735158120609807243406920622479028675861867188700142242008245503780134788615240070366814343778558954368227051103746151976471196298585353692960185063728102834229487303086437381579585610277271285378214113424669576506458723467835448641364705840083620190916949951521359560722259343077555697523316715494385988125564579036286712689559470988859604831601147492345725694662228200101
Calculated hash from m: 9d35b3d54154ed69cf1cdeb784f4a056572b0a3acf66e7b0d786d9c11f3a3993
Extracted hash of pad : 8e3e3153c850d6c972a6163497e158cd10fd67019e2b62acfa4ba68e25ac506d
Verify signature:  False
-------------------
Attack
-------------------
Matrice réduite : [ 10961736214113151133497950961356021494977398459513104272368781888596497297910487918148260543720782